# Generate new inputs v2

## Goal

On this second notebook the generation of inputs will be done using the `inference` script, I will use this notebook
to visualize the generated inputs and optimize the data and parameters.

## Imports

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import json
import sys
import os

sys.path.append(os.path.realpath('../scripts/'))
from evaluation import plot_grid
from arc24.data import load_arc_data_with_solutions

plt.plot()
plt.close('all')
plt.rcParams["figure.figsize"] = (25, 4)
mpl.rcParams['lines.linewidth'] = 3
mpl.rcParams['font.size'] = 16

## How to generate

```bash
python inference.py \
--model_path /mnt/hdd0/Kaggle/arc24/models/20240925_submission_models/06_continue-full-fine-tuning-Qwen2.5-0.5B-Instruct_lr1.5e-5_1e5steps_2gpus_8192msl/checkpoint-100000 \
--prompt_version input-from-inputs-v0 \
--dataset_path /mnt/hdd0/Kaggle/arc24/data/arc-agi_training_challenges.json \
--output_filepath /mnt/hdd0/Kaggle/arc24/input_generations/01_original_train_x8.json \
--predictions_per_task 8 \
--temperature 0.7 \
--n_tasks 10


python inference.py \
--model_path /mnt/hdd0/Kaggle/arc24/models/20240925_submission_models/06_continue-full-fine-tuning-Qwen2.5-0.5B-Instruct_lr1.5e-5_1e5steps_2gpus_8192msl/checkpoint-100000 \
--prompt_version input-from-inputs-v0 \
--dataset_path /mnt/hdd0/Kaggle/arc24/data/all_inputs/training.json \
--output_filepath /mnt/hdd0/Kaggle/arc24/input_generations/02_all_inputs_train_x8_temp0.json \
--predictions_per_task 8 \
--temperature 0 \
--n_tasks 10

python inference.py \
--model_path /mnt/hdd0/Kaggle/arc24/models/20240925_submission_models/06_continue-full-fine-tuning-Qwen2.5-0.5B-Instruct_lr1.5e-5_1e5steps_2gpus_8192msl/checkpoint-100000 \
--prompt_version input-from-inputs-v0 \
--dataset_path /mnt/hdd0/Kaggle/arc24/data/all_inputs/training.json \
--output_filepath /mnt/hdd0/Kaggle/arc24/input_generations/02_all_inputs_train_x8_temp5e-1.json \
--predictions_per_task 8 \
--temperature 0.5 \
--n_tasks 10


python inference.py \
--model_path /mnt/hdd0/Kaggle/arc24/models/20240925_submission_models/06_continue-full-fine-tuning-Qwen2.5-0.5B-Instruct_lr1.5e-5_1e5steps_2gpus_8192msl/checkpoint-100000 \
--prompt_version input-from-inputs-v0 \
--dataset_path /mnt/hdd0/Kaggle/arc24/data/all_inputs/training.json \
--output_filepath /mnt/hdd0/Kaggle/arc24/input_generations/02_all_inputs_train_x8_temp7e-1.json \
--predictions_per_task 8 \
--temperature 0.7 \
--n_tasks 10

python inference.py \
--model_path /mnt/hdd0/Kaggle/arc24/models/20240925_submission_models/06_continue-full-fine-tuning-Qwen2.5-0.5B-Instruct_lr1.5e-5_1e5steps_2gpus_8192msl/checkpoint-100000 \
--prompt_version input-from-inputs-v0 \
--dataset_path /mnt/hdd0/Kaggle/arc24/data/all_inputs/training.json \
--output_filepath /mnt/hdd0/Kaggle/arc24/input_generations/02_all_inputs_train_x8_temp9e-1.json \
--predictions_per_task 8 \
--temperature 0.9 \
--n_tasks 10
```

## Visualize generated inputs

In [ ]:
def visualize_generated_inputs(filepath):
    with open(filepath, 'r') as f:
        data = json.load(f)
    for task_id, task_generations in data.items():
        print(task_id)
        for sample_generations in task_generations:
            for plot_idx, generation in enumerate(sample_generations.values(), 1):
                plt.subplot(1, len(sample_generations), plot_idx)
                plot_grid(generation)
            plt.show()

In [ ]:
visualize_generated_inputs('/mnt/hdd0/Kaggle/arc24/input_generations/02_all_inputs_train_x8_temp0.json')

In [ ]:
visualize_generated_inputs('/mnt/hdd0/Kaggle/arc24/input_generations/02_all_inputs_train_x8_temp5e-1.json')

In [ ]:
visualize_generated_inputs('/mnt/hdd0/Kaggle/arc24/input_generations/02_all_inputs_train_x8_temp7e-1.json')

In [ ]:
visualize_generated_inputs('/mnt/hdd0/Kaggle/arc24/input_generations/02_all_inputs_train_x8_temp9e-1.json')

## Create dataset with all the inputs

Initial generations seem to overfit to the test input. If I add the test input to the inputs I will force the model to generate new inputs.

In [ ]:
training_data = load_arc_data_with_solutions('/mnt/hdd0/Kaggle/arc24/data/arc-agi_training_challenges.json')

In [ ]:
def create_all_inputs_dataset(dataset):
    all_inputs_dataset = {}
    for task_id, task in dataset.items():
        all_inputs_dataset[task_id] = dict(
            train=task['train'] + task['test'],
            test=task['test'][:1], # only one test sample
        )
    return all_inputs_dataset

In [ ]:
training_all_inputs_dataset = create_all_inputs_dataset(training_data)

In [ ]:
with open('/mnt/hdd0/Kaggle/arc24/data/all_inputs/training.json', 'w') as f:
    json.dump(training_all_inputs_dataset, f)